# Evaluate the impact of followign an ACC on the behaviour of a human driver

This notebook will allow you to reproduce the results that are presented in the paper <br>
#### Does following an ADAS change the behaviour of a driver ? <br>
Thibault Charlottin, Haneen Farah, Silvia Varotto, Christine Buisson

In [ ]:
#import the coded scripts

#data preprocessing tools
import src.read_data as read
import src.add_safety_indicators as safety
import src.detect_LC as LC
import src.add_stimulus_evaluation as stimulus
#longitudinal comparison tools
import src.cftest as cf
import src.compare_cf_behaviour as car_follow
import src.compare_cf_execution as cf_exe
import src.compare_stimulus_reaction as stim_compa

#safety comparison tool
import src.compare_safety as safety_compa

#LC probability
import src.prepare_data_for_LC_logit as LC_preprocessing
import src.model_LC_proba as Logit_LC

#misc packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os 

#plotting environnement
sns.set_theme()
sns.set(font_scale=1.2)

## Preprocess data

First we create one individual dataframe for each run and we add the leader, distance headway, time headway and traffic safety indicators and lane change flags <br>
To launch this cell, you need to download the TGSIM dataset as presented in the Readme file and then, as presented in the Readme, to place them into the "data/raw data/" folder

In [ ]:
#in a first time we perform leader detection and DHW and THW computations 
path = 'data/raw_data/'
path_out = 'data/by_run/'
filenames = ['Third_Generation_Simulation_Data__TGSIM__I-294_L2_Trajectories.csv',
'Third_Generation_Simulation_Data__TGSIM__I-90_I-94_Moving_Trajectories.csv',
'Third_Generation_Simulation_Data__TGSIM__I-90_I-94_Stationary_Trajectories.csv',
'Third_Generation_Simulation_Data__TGSIM__I-294_L1_Trajectories.csv']

location_names = ['294_L2_by_run/','90_94_moving_by_run/','90_94_static_by_run/','294_L1_by_run/']

for k in range(len(location_names)):
    read.clean_data(path+filenames[k], path_out+location_names[k])


In [ ]:
# then we perform LC detection

path_list = ['Data/by_run/294_L2_by_run/','Data/by_run/294_L1_by_run/',
             'Data/by_run/90_94_moving_by_run/','Data/by_run/90_94_static_by_run/']

for path in path_list:
    files = os.listdir(path)
    for f in files:
        df = pd.read_csv(path+f)
        df = LC.detect_lane_change(df)
        df.to_csv(path+f)


In [ ]:
# and compute traffic safety indicators 

for path in path_list:
    files = os.listdir(path)
    for f in files:
        df = pd.read_csv(path+f)
        safety.process_file(df,path_list)


In [ ]:
#before computing the speeddeltas
for path in path_list:
    files = os.listdir(path)
    for f in files:
        df = pd.read_csv(path+f)
        stimulus.add_stimulus_info(path+f)

## Impact of following an ADAS on the car-following execution

In [ ]:
# first we use the DTW algorithm to detect all the vehicles that are car-following

path_out = 'out/data/DTW/'
location_names = ['294_L2_by_run/','90_94_moving_by_run/','90_94_static_by_run/','294_L1_by_run/']
path_list = ['Data/by_run/294_L2_by_run/','Data/by_run/294_L1_by_run/',
             'Data/by_run/90_94_moving_by_run/','Data/by_run/90_94_static_by_run/']
window = 30
step = 5
tau = 1.2
for path in range(len(path_list)):
    for f in os.listdir(path_list[path]):
        df = pd.read_csv(path_list[path]+f)
        df_DTW = cf.create_dtw_by_time_df(df,tau,window,step)
        df_DTW.to_csv(path_out+location_names[path]+f)

In [ ]:
# we compare the impact of following an ADAS on the car-following behaviour

path_list = ['294_L1_by_run/','294_L2_by_run/','90_94_static_by_run/','90_94_moving_by_run/']
glob_dtw = 'out/data/DTW/'
glob_trajs = 'data/by_run/'

df_DTW = car_follow.merge_df_dtw(path_list,glob_dtw,glob_trajs)
df_DTW.to_csv('out/data/DTW/summary.csv',index=False)
results = car_follow.plot_CF_DTW(df_DTW)
results.to_csv('out/data/DTW/percentage_cf.csv')
car_follow.plot_speed_DTW(df_DTW)
teststats = car_follow.compare_groups_statistics(df_DTW)
ACC = df_DTW[(df_DTW['leader is ACC']==True)&(df_DTW['location']=='294_L1_by_run/')]
AV = df_DTW[(df_DTW['leader is ACC']==True)&(df_DTW['location']!='294_L1_by_run/')]
HDV = df_DTW[(df_DTW['leader is ACC']==False)]
result = ACC.groupby('id')['time'].agg(lambda x: x.max() - x.min()).reset_index()
result.rename(columns={'time': 'time_range'}, inplace=True)
print(result['time_range'].sum())
result = AV.groupby('id')['time'].agg(lambda x: x.max() - x.min()).reset_index()
result.rename(columns={'time': 'time_range'}, inplace=True)
print(result['time_range'].sum())
result = HDV.groupby('id')['time'].agg(lambda x: x.max() - x.min()).reset_index()
result.rename(columns={'time': 'time_range'}, inplace=True)
print(result['time_range'].sum())


In [ ]:
# we compare the impact of following an ADAS on the car-following execution

path_list = ['294_L1_by_run/','294_L2_by_run/','90_94_static_by_run/','90_94_moving_by_run/']
glob_trajs = 'data/by_run/'

trajs =  cf_exe.merge_df_trajs(path_list,glob_trajs)
results_df = cf_exe.compare_groups_statistics(trajs)
results_df.to_csv('out/tests/DHW_ranges_tests.csv')


trajs.replace([np.inf, -np.inf], np.nan, inplace=True)
cf_exe.plot_speed_DHW(trajs)


In [ ]:
# we compare the impact of following an ADAS on the response to a stimulus

path_list = ['294_L1_by_run/','90_94_static_by_run/','90_94_moving_by_run/']
glob_trajs = 'data/by_run/'

merged = stim_compa.merge_df_trajs(path_list,glob_trajs)
merged.reset_index(inplace = True, drop = True)
results_df = stim_compa.test_acceleration_differences(merged)
results_df.to_csv('out/tests/speedelta_test.csv')

stim_compa.plot_response_vs_stimulus(merged)

## Examining the impact of following an ADAS on traffic safety

In [ ]:

path_list = ['294_L1_by_run/','90_94_static_by_run/','90_94_moving_by_run/']
glob_trajs = 'data/by_run/'

merged = safety_compa.merge_df_trajs(path_list,glob_trajs)
safety_compa.follow_ACC(merged,22,'TTC')

## Examining the impact of following an ADAS on lane changing probability

the following cell can be bypassed, the intermediate result is already saved in the reposit

In [ ]:
#prepare data for the logit model

logit_df = pd.DataFrame()
path_list = ['294_L1_by_run/','294_L2_by_run/','90_94_static_by_run/','90_94_moving_by_run/']
glob_dtw = 'out/data/DTW/'
glob_trajs = 'data/by_run/'
for p in path_list:
    for f in os.listdir(glob_dtw+p):
        trajs = pd.read_csv(glob_trajs+p+f)
        dtw = pd.read_csv(glob_dtw+p+f)
        trajs = LC_preprocessing.compute_speed_diff(trajs)
        loc_logit = LC_preprocessing.construct_Logit_DF(trajs,dtw)
        loc_logit['location'] = [p for k in range(len(loc_logit['id']))]
        logit_df = pd.concat([logit_df,loc_logit])
logit_df.to_csv('out/data/LC_data/logit_df.csv')

In [ ]:
#model lane change

data_LC = Logit_LC.prepare_data(path)

logit_results = Logit_LC.logit_model(data_LC)

display(logit_results)